In [1]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional
import importlib
import backend.adapters.repository as repository
import backend.domain.model as model
import backend.service_layer.service as service
import backend.service_layer.uow as uow
from sec_api import XbrlApi
import pandas as pd
import os
from backend.domain.model import Company, Filing

c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
uow_instance = uow.UnitOfWork()
company = service.get_company_by_ticker('RDDT', uow_instance)


In [3]:
combined = service.get_consolidated_income_statements('RDDT', uow_instance, '10-Q')

Years covered in last filing (2025-05-02):
Index(['2024-01-01:2024-03-31', '2025-01-01:2025-03-31'], dtype='object', name='date_range')
Index(['2024-01-01:2024-03-31', '2025-01-01:2025-03-31'], dtype='object', name='date_range')
Index(['2024-01-01:2024-03-31', '2025-01-01:2025-03-31'], dtype='object', name='date_range')
loading filings for [<backend.domain.model.Filing object at 0x00000199D04E04D0>, <backend.domain.model.Filing object at 0x00000199D04E03D0>, <backend.domain.model.Filing object at 0x00000199D04E0490>, <backend.domain.model.Filing object at 0x00000199D04E0410>, <backend.domain.model.Filing object at 0x00000199D04E0F90>]
loading filings for 2024-12-31 10-K
loading filings for 2025-03-31 10-Q
loading filings for 2024-09-30 10-Q
loading filings for 2024-06-30 10-Q
loading filings for 2024-03-31 10-Q
Index(['2022-01-01:2022-12-31', '2023-01-01:2023-12-31',
       '2024-01-01:2024-12-31'],
      dtype='object', name='date_range')
Index(['2022-01-01:2022-12-31', '2023-01-01:20

c:\Users\Site Oracle\reduceComplexity\backend\domain\model.py:586: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  self.df = self.df.applymap(self.convert_to_millions)


In [4]:
combined.df

date_range,2023-01-01:2023-03-31,2023-04-01:2023-06-30,2023-07-01:2023-09-30,2023-10-01:2023-12-31,2024-01-01:2024-03-31,2024-04-01:2024-06-30,2024-07-01:2024-09-30,2024-10-01:2024-12-31,2025-01-01:2025-03-31
Cost of Goods Sold,27.00,29.0,26.00,29.00,28.00,30.00,35.00,32.00,37.00
Costs & Expenses,234.00,236.0,227.00,247.00,832.00,312.00,341.00,375.00,388.00
Basic EPS,-1.05,-0.7,-0.13,0.34,-8.19,-0.06,0.18,4.74,0.14
Diluted EPS,-1.05,-0.7,-0.13,0.34,-8.19,-0.06,0.16,4.76,0.13
G&A Expense,41.00,38.0,37.00,48.00,243.00,68.00,66.00,74.00,69.00
Operating Income,-60.00,-40.0,-7.00,19.00,-575.00,-10.00,30.00,70.00,24.00
Income Tax,1.00,1.0,0.00,1.00,0.00,0.00,-31000.00,-1.00,-2.00
Net Income,-61.00,-41.0,-7.00,19.00,-575.00,-10.00,30.00,71.00,26.00
Other Income (Expense),11.00,13.0,13.00,16.00,15.00,21.00,23.00,17.00,21.00
Operating Income,-71.00,-53.0,-20.00,3.00,-589.00,-31.00,7.00,53.00,4.00


In [5]:
def convert_to_millions(val):
    try:
        # Check if it's a numeric value
        num = float(val)
        
        # Only convert to millions if number is at least 100,000
        if abs(num) >= 100_000:
            return round(num / 1000000, 2)
        else:
            return num
    except (ValueError, TypeError):
        # Return original value if not numeric
        return val

In [6]:
def is_sparse_row(row):
    # Count zero values in the row
    numeric_values = 0
    zero_values = 0
    
    for val in row:
        try:
            num = float(val)
            numeric_values += 1
            if num == 0:
                zero_values += 1
        except (ValueError, TypeError):
            # Skip non-numeric values
            pass
    
    # If no numeric values, don't delete
    if numeric_values == 0:
        return False
        
    # Calculate percentage of zeros
    zero_percentage = zero_values / numeric_values if numeric_values > 0 else 0
    
    # Return True if more than 50% zeros
    return zero_percentage > 0.5

In [7]:
combined.df = combined.df.applymap(convert_to_millions)


C:\Users\Site Oracle\AppData\Local\Temp\ipykernel_34160\1361744653.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined.df = combined.df.applymap(convert_to_millions)


In [8]:
combined.df = combined.df.loc[[not is_sparse_row(row) for _, row in combined.df.iterrows()]]

In [9]:
combined.df

date_range,2023-01-01:2023-03-31,2023-04-01:2023-06-30,2023-07-01:2023-09-30,2023-10-01:2023-12-31,2024-01-01:2024-03-31,2024-04-01:2024-06-30,2024-07-01:2024-09-30,2024-10-01:2024-12-31,2025-01-01:2025-03-31
Cost of Goods Sold,27.00,29.0,26.00,29.00,28.00,30.00,35.00,32.00,37.00
Costs & Expenses,234.00,236.0,227.00,247.00,832.00,312.00,341.00,375.00,388.00
Basic EPS,-1.05,-0.7,-0.13,0.34,-8.19,-0.06,0.18,4.74,0.14
Diluted EPS,-1.05,-0.7,-0.13,0.34,-8.19,-0.06,0.16,4.76,0.13
G&A Expense,41.00,38.0,37.00,48.00,243.00,68.00,66.00,74.00,69.00
Operating Income,-60.00,-40.0,-7.00,19.00,-575.00,-10.00,30.00,70.00,24.00
Income Tax,1.00,1.0,0.00,1.00,0.00,0.00,-31000.00,-1.00,-2.00
Net Income,-61.00,-41.0,-7.00,19.00,-575.00,-10.00,30.00,71.00,26.00
Other Income (Expense),11.00,13.0,13.00,16.00,15.00,21.00,23.00,17.00,21.00
Operating Income,-71.00,-53.0,-20.00,3.00,-589.00,-31.00,7.00,53.00,4.00


In [10]:
combined.analyze_period_coverage()

{'coverage_by_year': {2023: ['Q1', 'Q2', 'Q3', 'Q4'],
  2024: ['Q1', 'Q2', 'Q3', 'Q4'],
  2025: ['Q1']},
 'missing_periods': {2025: ['Q2', 'Q3', 'Q4']},
 'has_continuous_coverage': True,
 'years_analyzed': [2023, 2024, 2025]}

In [11]:
for filing in combined.source_filings:
    try: 
        print(filing.cover_page.document_fiscal_period_focus, filing.cover_page.document_fiscal_year_focus)
    except:
        print(filing.form)

FY 2024
Q1 2025
Q3 2024
Q2 2024
Q1 2024


In [12]:
sorted_columns = sorted(combined.df.columns, key=lambda x: x.split(':')[0])
print(sorted_columns)

['2023-01-01:2023-03-31', '2023-04-01:2023-06-30', '2023-07-01:2023-09-30', '2023-10-01:2023-12-31', '2024-01-01:2024-03-31', '2024-04-01:2024-06-30', '2024-07-01:2024-09-30', '2024-10-01:2024-12-31', '2025-01-01:2025-03-31']


In [13]:
# Get data for first 4 filings
for filing in company.filings[:1]:
    filing.data = uow_instance.sec_filings.get_filing_data(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    filing.filing_url = uow_instance.sec_filings.get_filing_url(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')


Loaded data for filing date 2025-05-02, form 10-Q


In [14]:
res = company.select_filings_with_processing_pattern(company.filings,'10-Q')

AttributeError: 'NoneType' object has no attribute 'table'

In [31]:
for filing in res:
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')

Loaded data for filing date 2024-12-06, form 10-Q
Loaded data for filing date 2024-09-06, form 10-Q
Loaded data for filing date 2024-06-07, form 10-Q
Loaded data for filing date 2022-12-02, form 10-Q
Loaded data for filing date 2022-09-02, form 10-Q
Loaded data for filing date 2022-06-02, form 10-Q
Loaded data for filing date 2020-12-04, form 10-Q
Loaded data for filing date 2020-09-04, form 10-Q
Loaded data for filing date 2020-06-11, form 10-Q
Loaded data for filing date 2018-12-07, form 10-Q
Loaded data for filing date 2018-09-06, form 10-Q
Loaded data for filing date 2018-06-07, form 10-Q
Loaded data for filing date 2016-12-06, form 10-Q
Loaded data for filing date 2016-09-01, form 10-Q
Loaded data for filing date 2016-06-02, form 10-Q
Loaded data for filing date 2014-12-09, form 10-Q
Loaded data for filing date 2014-09-05, form 10-Q
Loaded data for filing date 2014-06-06, form 10-Q
Loaded data for filing date 2012-12-05, form 10-Q
Loaded data for filing date 2012-09-05, form 10-Q
